
# 🚀 uv Super-Notebook: instalación, versiones de Python, entornos, deps y más

Este notebook es un **ejecutable** para trabajar con **[uv](https://docs.astral.sh/uv/)**: instalarlo, gestionar **múltiples versiones de Python**, fijar una versión **por proyecto**, inicializar proyectos, **sync**, **add/remove**, **uvx**, exportar `requirements.txt`, troubleshooting, y migración desde `pyenv/Poetry`.

> **Nota:** Los comandos de shell están puestos como ejemplo. Puedes ejecutarlos desde una terminal del sistema. Aquí los verás listados para copiar/pegar.



## 1) Instalar **uv**

Instala el binario de `uv` (Linux/WSL). Si ya lo tienes, salta este paso.


In [ ]:

# Linux/WSL
# Ejecutar en tu terminal (no dentro del notebook si no tienes permisos):
# curl -LsSf https://astral.sh/uv/install.sh | sh
# exec $SHELL -l
# uv --version



## 2) Instalar **versiones de Python** con uv

`uv` instala runtimes de Python **aislados por usuario** (sin compilar). No ensucia tu sistema.


In [ ]:

# Instalar versiones específicas
# uv python install 3.12
# uv python install 3.13

# Ver versiones disponibles/instaladas
# uv python list

# Encontrar la ruta exacta de una versión
# uv python find 3.12
# uv python find 3.13



## 3) Elegir una versión de Python **por proyecto**

Dentro del *root* del proyecto, fija (pin) una versión de Python. Esto crea `.python-version`.


In [ ]:

# En el directorio del proyecto:
# uv python pin 3.12
# uv python pin           # muestra la versión pineada

# En otro proyecto, puedes pinear otra versión:
# uv python pin 3.13

# uv run python -c "import sys; print(sys.executable)" --> valida la ruta donde quedo instalado el python, por si se requiere el path para crear un env y seleccionarlo con un notebook



## 4) Inicializar un proyecto con uv

Crea estructura base y `pyproject.toml`.


In [ ]:

# Proyecto nuevo
# uv init mi_proyecto
# cd mi_proyecto
# uv init --src .   # opcional: layout con carpeta src

# Proyecto existente (en el root del repo)
# uv init .



## 5) Crear/usar entorno virtual y **sincronizar dependencias**

Con `uv` no necesitas `python -m venv` (aunque también puedes). Lo usual:


In [ ]:

# Camino recomendado (automático)
# Primero fija la versión del proyecto (ver sección 3).
# uv sync
# - Crea .venv si no existe
# - Instala deps según pyproject/uv.lock

# Camino manual (si prefieres)
# uv venv --python 3.12   # crea venv explícito con esa versión
# uv sync                  # resuelve e instala deps



## 6) Añadir / quitar dependencias

`uv add` y `uv remove` gestionan tu `pyproject.toml` y el `uv.lock`.


In [ ]:

# Añadir dependencias de runtime
# uv add boto3 pandas awswrangler

# Añadir dependencias de desarrollo
# uv add --dev pytest ruff black  --> "uv add --dev" es atajo de "uv run --group dev"
 
# ejemplo: uv add --dev ipykernel --> CUando se trabaja con notebooks

# Quitar dependencias
# uv remove pandas



## 7) Ejecutar con el entorno del proyecto (`uv run`)


In [ ]:

# Ejecutar un script
# uv run python src/main.py

# Ejecutar un módulo
# uv run -m paquete.modulo

# Definir scripts en pyproject.toml y correrlos por nombre
# uv run tests



## 8) Lock y visualización de dependencias


In [ ]:

# Recalcular lock después de cambiar rangos en pyproject
# uv lock

# Ver árbol de dependencias resueltas
# uv tree



## 9) Exportar a `requirements.txt` (compatibilidad con otros)


In [ ]:

# Exportar dependencias bloqueadas
# uv export -o requirements.txt



## 10) **uvx**: ejecutar herramientas/CLIs al vuelo (sin instalaciones globales)


In [ ]:

# Ejecutar ruff, black, httpie, etc. sin instalarlos globalmente
# uvx ruff --version
# uvx black --help
# uvx http --help
# uvx ruff check scripts/connect-llm.py --> un archivo en especifico
# uvx ruff check --fix scripts/connect-llm.py --> un archivo en especifico
# uvx ruff format scripts/connect-llm.py --> un archivo en especifico

# uv run ruff check scripts/trying-different-llm-models.py


# uv run vs uvx

uv run <cmd>: ejecuta el comando dentro del .venv del proyecto, respetando tu uv.lock.
Ejemplos:

uv run python script.py
uv run pytest -q
uv run jupyter lab


Ideal cuando la herramienta está declarada en el proyecto (prod o dev).

uvx <tool>: ejecuta una herramienta efímera (estilo pipx run): la descarga si hace falta, la cachea y la corre fuera de tu proyecto (no toca pyproject.toml ni uv.lock).
Ejemplos:

uvx ruff format .
uvx --from jupyterlab==4.2.5 jupyter-lab


## 11) Ejemplos multi-proyecto (cada uno con Python distinto)

Supón que tienes dos repos:
- `repo-a` con **Python 3.12**
- `repo-b` con **Python 3.13**


In [ ]:

# repo-a
# cd ~/proyectos/repo-a
# uv python install 3.12          # una sola vez
# uv python pin 3.12
# uv init .
# uv add boto3 pandas
# uv sync
# uv run python -V                 # 3.12.x

# repo-b
# cd ~/proyectos/repo-b
# uv python install 3.13          # una sola vez
# uv python pin 3.13
# uv init .
# uv add fastapi uvicorn
# uv sync
# uv run python -V                 # 3.13.x



## 12) Caché, configuración y housekeeping


In [ ]:

# Ubicar y limpiar caché
# uv cache dir
# uv cache prune

# Mostrar info del entorno actual
# uv info

# Forzar reinstalación de entornos si cambiaste de versión mayor
# uv sync --reinstall



## 13) Troubleshooting común

- **"No toma la versión que pinée"**: confirma que estás en el **root del proyecto**; revisa `.python-version`; ejecuta `uv python pin` y `uv sync`.  
- **"Se mezclan entornos"**: usa siempre `uv run ...` para ejecutar con el venv correcto.  
- **"Cambio de 3.12 a 3.13 y quedó raro"**: `uv sync --reinstall`.  
- **"Necesito compilar un paquete con C headers"**: instala los dev packages del sistema (ej. `build-essential`, `python3-dev`, `libssl-dev`, `libffi-dev`, `libsqlite3-dev`, etc.).  
- **"Quiero usar un venv en otra ruta"**: `uv venv --python 3.12 /ruta/a/mi-venv` y luego `uv run --python /ruta/a/mi-venv/bin/python ...` (avanzado).



## 14) Migración rápida desde pyenv / Poetry

**Desde pyenv:**  
- Mantén pyenv si lo prefieres, pero con `uv` ya no es necesario.  
- Reemplaza `pyenv local` por `uv python pin` y usa `uv sync` para crear `.venv` + deps.

**Desde Poetry:**  
- `uv init .` para crear/ajustar `pyproject.toml`.  
- Usa `uv add ...` para añadir deps en vez de `poetry add`.  
- Usa `uv sync` (reemplaza `poetry install`) y `uv run` (reemplaza `poetry run`).  
- Si necesitas exportar: `uv export -o requirements.txt`.



## 15) Notas para Windows/WSL

- En WSL (Ubuntu) funciona perfecto con el instalador de `uv`.  
- Si trabajas en Windows nativo, consulta la doc oficial para el instalador recomendado.  
- Evita mezclar `pip` global y `uv`. Si lo necesitas, usa `uv pip ...`.



## 16) Plantilla mínima de `pyproject.toml` con scripts

Ejemplo listo para copiar:



```toml
[project]
name = "mi-proyecto"
version = "0.1.0"
description = "Proyecto de ejemplo con uv"
readme = "README.md"
requires-python = ">=3.12"
dependencies = [
  "boto3",
  "pandas",
]

[project.optional-dependencies]
dev = [
  "pytest",
  "ruff",
]

[tool.uv]
# Config opcional para uv

[tool.uv.scripts]
tests = "pytest -q"
lint = "ruff check ."
format = "ruff format ."
```



---

### ✅ Resumen de flujo recomendado
```bash
cd ~/proyectos/mi-repo
uv python install 3.12       # una vez
uv python pin 3.12           # por proyecto
uv init .                    # crea/ajusta pyproject
uv add boto3 pandas          # añadir deps
uv sync                      # crea .venv e instala
uv run python src/main.py    # ejecutar con el venv correcto
```
¿Quieres que lo adapte a tu estructura de repos actual (p. ej., `lla-len-mul` y lambdas)? Puedo personalizarlo.
